In [2]:
#dependents
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
from scipy import stats

#create imporsonment data url
Imprisonment_baseUrl = "https://api.ed.gov/data/mbk-imprisonment?"
Imprisonment_apiKey = "api_key=DTeVODnGRjLME8jd6PQCOy1DpgLqXPdHWtGO4EqA"
Imprisonment_perPage = "per_page=" + "195"
Imprisonment_url = Imprisonment_baseUrl + Imprisonment_apiKey + "&" + Imprisonment_perPage

#request and get data as "ImprisonmentData" through url
ImprisonmentData = requests.get(Imprisonment_url).json()

#get length of imprisonment dataframe entries 
Imprisonment_length = len(ImprisonmentData["resources"])

#initialize lists for data pull from request
YearList = [] 
RaceEthnicityList = []
SexLists = []
CountList = []
CountPerCapitaList = []

#pull all data from requested data and put into lists for each column
for i in range (Imprisonment_length):
    YearList.append(int(ImprisonmentData["resources"][i]["Year"]))
    RaceEthnicityList.append(ImprisonmentData["resources"][i]["Race/ethnicity"])
    SexLists.append(ImprisonmentData["resources"][i]["Sex"])
    CountList.append(int(ImprisonmentData["resources"][i]["Count"]))
    CountPerCapitaList.append(int(ImprisonmentData["resources"][i]["Rate per 100,000"]))

#create dataframe and input all lists
imprisonmentDf = pd.DataFrame({
    "Year" : YearList,
    "Race/Ethnicity" : RaceEthnicityList,
    "Sex" : SexLists,
    "Count" : CountList,
    "Per Capita Rate per 100,000" : CountPerCapitaList
})

#show imprisonment dataframe
imprisonmentDf.sort_values(by=["Year", "Race/Ethnicity"], ascending=True).head(16)


,Year,Race/Ethnicity,Sex,Count,"Per Capita Rate per 100,000"
52,2000,"Black, non-Hispanic",None,104880,2732
104,2000,"Black, non-Hispanic",Male,101521,5358
156,2000,"Black, non-Hispanic",Female,3500,180
65,2000,Hispanic,None,48696,1004
117,2000,Hispanic,Male,46949,1765
169,2000,Hispanic,Female,1596,73
78,2000,"Other race, non-Hispanic",None,15350,826
130,2000,"Other race, non-Hispanic",Male,14504,1557
182,2000,"Other race, non-Hispanic",Female,869,94
39,2000,"White, non-Hispanic",None,59129,345


In [3]:
#replacing all races with standard races
imprisonmentDf = imprisonmentDf.replace("White, non-Hispanic", "Caucasian")
imprisonmentDf = imprisonmentDf.replace("Black, non-Hispanic", "African American")
imprisonmentDf = imprisonmentDf.replace("Other race, non-Hispanic", "Other")
imprisonmentDf.sort_values(by=["Year", "Race/Ethnicity"], ascending=True).head(16)


,Year,Race/Ethnicity,Sex,Count,"Per Capita Rate per 100,000"
52,2000,African American,None,104880,2732
104,2000,African American,Male,101521,5358
156,2000,African American,Female,3500,180
39,2000,Caucasian,None,59129,345
91,2000,Caucasian,Male,55602,639
143,2000,Caucasian,Female,3498,41
65,2000,Hispanic,None,48696,1004
117,2000,Hispanic,Male,46949,1765
169,2000,Hispanic,Female,1596,73
78,2000,Other,None,15350,826


In [5]:
#CREATE LIST OF RACES
raceList = ["Caucasian", "African American", "Hispanic", "Other"]
raceList

['Caucasian', 'African American', 'Hispanic', 'Other']

In [6]:
#Pull CSV - Drop Out Data
dropOut_df = pd.read_csv("../CSVs/Total_drop_out_rates_by_year_and_race.csv")
dropOut_df = dropOut_df.rename(columns={"Dropout White" : "Caucasian", "Dropout Black" : "African American", "Dropout Hispanic" : "Hispanic", "Dropout other" : "Other"})
dropOut_df = dropOut_df.set_index("Year")
#reorder columns
dropOut_df = dropOut_df[raceList]

#Pull CSV - 2000-2007
dropOut_00_07_df = pd.read_csv("../CSVs/Total_drop_out_rates_by_race(2000-2007).csv")
dropOut_00_07_df = dropOut_00_07_df.rename(columns={"Dropout White" : "Caucasian", "Dropout Black" : "African American", "Dropout Hispanic" : "Hispanic", "Dropout other" : "Other"})
dropOut_00_07_df = dropOut_00_07_df.set_index("Year")
#reorder columns
dropOut_00_07_df = dropOut_00_07_df[raceList]

#Pull CSV - Drop Out Data
dropOut_08Plus_df = pd.read_csv("../CSVs/Total_drop_out_rates_by_race(2008+).csv")
dropOut_08Plus_df = dropOut_08Plus_df.rename(columns={"Dropout White" : "Caucasian", "Dropout Black" : "African American", "Dropout Hispanic" : "Hispanic", "Dropout other" : "Other"})
dropOut_08Plus_df = dropOut_08Plus_df.set_index("Year")
#reorder columns
dropOut_08Plus_df = dropOut_08Plus_df[raceList]

In [ ]:
#IMPRISONMENTS TOTAL COUNT FOR ALL RACES AND ALL GENDERS
imprisonmentDf_totalImprisonments = pd.DataFrame(imprisonmentDf.groupby(["Year"])["Count"].sum())
imprisonmentGraph_totalImprisonments = imprisonmentDf_totalImprisonments.unstack(level=-1).plot(kind="line", legend=False)
imprisonmentGraph_totalImprisonments.set_ylabel("Total Imprisonments")
imprisonmentGraph_totalImprisonments.set_xlabel("Year")
plt.show()

In [ ]:
#IMPRISONMENTS TOTAL COUNT FOR ALL RACES AND ALL GENDERS - PER CAPITA
imprisonmentDf_totalImprisonmentsPerCapita = pd.DataFrame(imprisonmentDf.groupby(["Year"])["Per Capita Rate per 100,000"].sum()/100000*100)
imprisonmentGraph_totalImprisonmentsPerCapita = imprisonmentDf_totalImprisonmentsPerCapita.unstack(level=-1).plot(kind="line", legend=False)
imprisonmentGraph_totalImprisonmentsPerCapita.set_ylabel("Per Capita Rate per 100,000")
imprisonmentGraph_totalImprisonmentsPerCapita.set_xlabel("Year")
plt.show()

In [ ]:
#GRAPH IMPRISONMENTS PER RACE

In [ ]:
#GRAPH TOTAL IMPRISONMENTS PER RACE
#get imprisonment count per race
#Adds gender counts together into single race count
#unstacks -1 and then stacks 0
imprisonmentDf_total_PerRace = pd.DataFrame(imprisonmentDf.groupby(["Year", "Race/Ethnicity"])["Count"].sum()).unstack(level=-1).stack(level=0) 
imprisonmentDf_total_PerRace.reset_index(level=0, inplace=True)
imprisonmentDf_total_PerRace = imprisonmentDf_total_PerRace.set_index('Year')
imprisonmentDf_total_PerRace = imprisonmentDf_total_PerRace.rename_axis("", axis="columns")
imprisonmentDf_total_PerRace

#write CSV
imprisonmentDf_total_PerRace.to_csv("../CSVs/imprisonmentDf_total_PerRace.csv")

#plot
for i in imprisonmentRaceList:
    plt.plot(imprisonmentDf_total_PerRace.index, imprisonmentDf_total_PerRace[i])

#format
plt.legend(labels=imprisonmentDf_total_PerRace.columns, loc="best", markerscale=.5)
plt.xlabel("Year")
plt.ylabel("Imprisonments Per Race - Total")
plt.title("Imprisonments Per Race - Total between 2000 and 2012")
plt.savefig('../Graphs/Imprisonments Per Race - Total between 2000 and 2012.png')
#imprisonmentDf_total_PerRace

In [ ]:
#GRAPH TOTAL IMPRISONMENTS PER RACE PER CAPITA %
#get imprisonment count per race
#Adds gender counts together into single race count
#unstacks -1 and then stacks 0
imprisonmentDf_perRace_perCapita = pd.DataFrame(imprisonmentDf.groupby(["Year", "Race/Ethnicity"])["Per Capita Rate per 100,000"].sum()).unstack(level=-1).stack(level=0) 
imprisonmentDf_perRace_perCapita.reset_index(level=0, inplace=True)
imprisonmentDf_perRace_perCapita = imprisonmentDf_perRace_perCapita.set_index('Year')
imprisonmentDf_perRace_perCapita = imprisonmentDf_perRace_perCapita.rename_axis("", axis="columns")
imprisonmentDf_perRace_perCapita

#write CSV
imprisonmentDf_perRace_perCapita.to_csv("../CSVs/imprisonmentDf_perRace_perCapita.csv")

#plot
for i in imprisonmentRaceList:
    plt.plot(imprisonmentDf_perRace_perCapita.index, imprisonmentDf_perRace_perCapita[i])

#format
plt.legend(labels=imprisonmentDf_perRace_perCapita.columns, loc="best", markerscale=.5)
plt.xlabel("Year")
plt.ylabel("Imprisonments Per Race Per Capita")
plt.title("Imprisonments Per Race Per Capita - Total between 2000 and 2012")
plt.savefig('../Graphs/Imprisonments Per Race Per Capita - Total between 2000 and 2012.png')
#imprisonmentDf_perRace_perCapita

In [ ]:
#GRAPH TOTAL IMPRISONMENTS PER RACE PER CAPITA %
#get imprisonment count per race
#Adds gender counts together into single race count
#unstacks -1 and then stacks 0
imprisonmentDf_perRace_perCapita = pd.DataFrame(imprisonmentDf.groupby(["Year", "Race/Ethnicity"])["Per Capita Rate per 100,000"].sum()).unstack(level=-1).stack(level=0) 
imprisonmentDf_perRace_perCapita.reset_index(level=0, inplace=True)
imprisonmentDf_perRace_perCapita = imprisonmentDf_perRace_perCapita.set_index('Year')
imprisonmentDf_perRace_perCapita = imprisonmentDf_perRace_perCapita.rename_axis("", axis="columns")
imprisonmentDf_perRace_perCapita

#write CSV
imprisonmentDf_perRace_perCapita.to_csv("../CSVs/imprisonmentDf_perRace_perCapita.csv")

for i in imprisonmentRaceList:
    plt.plot(imprisonmentDf_perRace_perCapita.index, imprisonmentDf_perRace_perCapita[i]/1000)
    
plt.legend(labels=imprisonmentDf_perRace_perCapita.columns, loc="best", markerscale=.5)
plt.xlabel("Year")
plt.ylabel("Imprisonments Per Race Per Capita (%)")
plt.title("Imprisonments Per Race Per Capita (%) - Total between 2000 and 2012")
plt.savefig('../Graphs/Imprisonments Per Race Per Capita (%) - Total between 2000 and 2012.png')

In [ ]:
#GRAPH TOTAL IMPRISONMENTS VS TOTAL DROPOUTS

In [ ]:
#IMPRISONMENTS TOTAL COUNT FOR ALL RACES AND ALL GENDERS COMBINED
imprisonmentDf_totalImprisonments = pd.DataFrame(imprisonmentDf.groupby(["Year"])["Count"].sum())
plt.plot(imprisonmentDf_totalImprisonments.index, imprisonmentDf_totalImprisonments["Count"],)
plt.xlabel("Year")
plt.ylabel("Imprisonments Per Race Per Capita (%)")
plt.title("Imprisonments Per Race Per Capita (%) - Total between 2000 and 2012")
plt.savefig('../Graphs/Imprisonments Total - Total between 2000 and 2012.png')


In [ ]:
#IMPRINSONMENTS TOTAL VS PER CAPITA - ALL 4

for i in imprisonmentRaceList:
    saveTitle = "Imprisonment Total vs Per Capita - " + i
    saveDestination = "../Graphs/" + saveTitle
    
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Total Imprisonment Count ', color=color)
    ax1.plot(imprisonmentDf_total_PerRace.index, imprisonmentDf_total_PerRace[i], color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    
    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('Total Imprisonment Count Per Capita', color=color)  # we already handled the x-label with ax1
    ax2.plot(imprisonmentDf_perRace_perCapita.index, imprisonmentDf_perRace_perCapita[i], color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.title(saveTitle)

    fig.savefig(saveDestination)

In [ ]:
#IMPRISONMENTS VS DROPOUTS - ALL 4
for i in range(len(imprisonmentRaceList)):
    saveTitle = "Imprisonment vs Dropout Per Race - " + imprisonmentRaceList[i]
    saveDestination = "../Graphs/" + saveTitle

    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Imprisonment for Hispanic % ', color=color)
    ax1.plot(imprisonmentDf_perRace_perCapita.index, imprisonmentDf_perRace_perCapita[imprisonmentRaceList[i]]/1000, color=color)
    ax1.tick_params(axis='y', labelcolor=color)


    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('Dropout Hispanic%', color=color)  # we already handled the x-label with ax1
    ax2.plot(dropOut_df.index, dropOut_df[dropOutRaceList[i]], color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    plt.title(saveTitle)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped

    fig.savefig(saveDestination)

In [ ]:
for i in range(len(imprisonmentRaceList)):
    saveTitle = "Imprisonment vs Dropout-  Scatter - " + imprisonmentRaceList[i]
    saveDestination = "../Graphs/" + saveTitle

    #plot scatter
    plt.scatter(dropOut_df[dropOutRaceList[i]], imprisonmentDf_perRace_perCapita[imprisonmentRaceList[i]]/1000)
    #create linregress
    slope, intercept, r_value, p_value, std_err = stats.linregress(dropOut_df[dropOutRaceList[i]], imprisonmentDf_perRace_perCapita[imprisonmentRaceList[i]]/1000)
    #plot line of best fit
    plt.plot(dropOut_df[dropOutRaceList[i]], intercept + slope*dropOut_df[dropOutRaceList[i]], 'r', label='fitted line')

    #format graph
    xlabel = "Dropout Rate for " + imprisonmentRaceList[i] + "(%)"
    plt.xlabel(xlabel)
    ylabel = "Imprisonments Rate for " + imprisonmentRaceList[i] + "(%)"
    plt.ylabel(ylabel)
    plt.legend(labels=imprisonmentDf_perRace_perCapita.columns, loc="best", markerscale=.5)
    #plt.text(0, 0, r_value**2, fontdict=None, withdash=False)

    plt.title(saveTitle)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped

    plt.show()
    #print(r_value)

In [ ]:

plt.xlabel("Dropout Rate for African American (%)")
plt.ylabel("Imprisonments Rate for African Americans (%)")
plt.legend(labels=imprisonmentDf_perRace_perCapita.columns, loc="best", markerscale=.5)
slope, intercept, r_value, p_value, std_err = stats.linregress(dropOut_df["Dropout Black"], imprisonmentDf_perRace_perCapita["Black, non-Hispanic"]/1000)
plt.plot(dropOut_df["Dropout Black"], intercept + slope*dropOut_df["Dropout Black"], 'r', label='fitted line')
plt.show()
print(r_value**2)

In [ ]:
plt.scatter(dropOut_df["Dropout Hispanic"], imprisonmentDf_perRace_perCapita["Hispanic"]/1000)

plt.xlabel("Dropout Rate for Hispanic (%)")
plt.ylabel("Imprisonments Rate for Hispanic (%)")
plt.legend(labels=imprisonmentDf_perRace_perCapita.columns, loc="best", markerscale=.5)
slope, intercept, r_value, p_value, std_err = stats.linregress(dropOut_df["Dropout Hispanic"], imprisonmentDf_perRace_perCapita["Hispanic"]/1000)
plt.plot(dropOut_df["Dropout Hispanic"], intercept + slope*dropOut_df["Dropout Hispanic"], 'r', label='fitted line')
plt.show()
print(r_value**2)